In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

import scipy.sparse
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression as lm
from scipy import stats

import gensim
import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, punkt
from gensim.models import KeyedVectors

import matplotlib.collections as plt1
import matplotlib.pyplot as plt2
from matplotlib.legend_handler import HandlerLineCollection, HandlerTuple
from matplotlib.lines import Line2D
import seaborn as sns

import pandas as pd
pd.set_option('display.max_columns', 50)

import pickle
import time
import re
import csv
import math
import pickle
import time
import spacy

/apps/software/standard/core/anaconda/2020.11-py3.8/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


# Data from supervised classification

In [2]:
# Compute a score to be identify as big data
df_merge = pd.read_csv("/project/biocomplexity/sdad/projects_data/ncses/prd/Digital_abstract_labelled/FR_final_predicted.csv")
df_merge['score'] = np.where(df_merge['Prediction_KNN'].str.contains("Non Big-data"), 0, 1) + np.where(df_merge['Prediction_SVM'].str.contains("Non Big-data"), 0, 1) + np.where(df_merge['Prediction_DT'].str.contains("Non Big-data"), 0, 1) + np.where(df_merge['Prediction_RF'].str.contains("Non Big-data"), 0, 1) + np.where(df_merge['Prediction_NN'].str.contains("Non Big-data"), 0, 1)
df_merge.head()

/apps/software/standard/core/anaconda/2020.11-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (3,7,8,11,12,13,14,16,17,18,19,20,21,22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,PROJECT_ID,ABSTRACT,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,FY_TOTAL_COST_SUM,NUM_RECORDS,final_tokens,index,Is it related to Big-Data,label,Prediction_KNN,Prediction_SVM,Prediction_DT,Prediction_RF,Prediction_NN,score
0,0,89996,"This is a project to explore Game-based, Metap...",Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,8/31/2012,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",1.0,068719400,WHEELING JESUIT UNIVERSITY,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008,1999467.0,NaN,1,1,1999467.0,1,project explore game base metaphor enhanced ga...,0,NaN,NaN,Big-data,Non Big-data,Non Big-data,Non Big-data,Non Big-data,1
1,1,89997,Institution: Franklin Institute Science Museum...,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,8/31/2012,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",2.0,001741859,FRANKLIN INSTITUTE,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008,1799699.0,NaN,1,1,1799699.0,1,institution franklin institute science museum ...,1,NaN,NaN,Big-data,Non Big-data,Non Big-data,Non Big-data,Big-data,2
2,2,89998,Through programs (including small group conver...,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,8/31/2011,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",4.0,061451670,SCIENCE MUSEUM OF MINNESOTA,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008,1505858.0,NaN,1,1,1505858.0,1,program include small group conversation citiz...,2,NaN,NaN,Big-data,Non Big-data,Non Big-data,Non Big-data,Non Big-data,1
3,3,89999,In partnership with the American Chemical Soci...,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,12/31/2010,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",4.0,009059242,INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008,51000.0,NaN,1,1,51000.0,1,partnership american chemical society acs nati...,3,NaN,NaN,Non Big-data,Non Big-data,Non Big-data,Non Big-data,Non Big-data,0
4,4,90001,The Center for Molecular Interfacing (CMI) wil...,Address; Architecture; Carbon Nanotubes; Catal...,CCI PHASE I: CENTER FOR MOLECULAR INTERFACING,NSF,NSF,NaN,0847926,10/1/2008,9/30/2011,"ABRUNA, HECTOR D",NaN,22.0,872612445,CORNELL UNIVERSITY ITHACA,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.049,2008,1519821.0,NaN,1,1,1519821.0,1,center molecular interfacing cmi enable integr...,4,NaN,NaN,Big-data,Non Big-data,Non Big-data,Big-data,Non Big-data,2


In [3]:
# Distribution of abstracts
df_merge['score'].value_counts()

0    738966
1    287333
2     77573
3     23219
4     11514
5      5264
Name: score, dtype: int64

In [4]:
# Used a rule (majority votes from classifiers) to identify Big-data abstracts. Useful because we have an impair number of classifier
df_merge['Supervised'] = 0
df_merge.loc[df_merge['score']>2,'Supervised'] = 1
df_merge['Supervised'].value_counts()

0    1103872
1      39997
Name: Supervised, dtype: int64

# Add the unsupervised classification

## 1. Term matching using key words: big data, big datum, big_data

In [5]:
# identify abstract with the word big-data
docs = df_merge["final_tokens"]

term_match_data = []

# big_datum: 0 matches
for i, abstract in enumerate(docs):
        if ('big datum' in abstract or 'big data' in abstract or 'big_data' in abstract):
            term_match_data.append(i)
            
# print
print('Number of abstracts with the word big-data:',len(term_match_data))

Number of abstracts with the word big-data: 4684


In [7]:
# unsupervised classification using term matching
df = df_merge
df['Unsupervised_term_match'] = 0
df.loc[term_match_data,'Unsupervised_term_match'] =1

In [17]:
# Comparison table between supervised and unsupervised (Overall)
pd.crosstab(df['Unsupervised_term_match'], df['Supervised'], margins=True, margins_name="Total", normalize='index')

Supervised,0,1
Unsupervised_term_match,,
0,0.967967,0.032033
1,0.251708,0.748292
Total,0.965034,0.034966


In [18]:
# Comparison with KNN
pd.crosstab(df['Unsupervised_term_match'], df['Prediction_KNN'], margins=True, margins_name="Total", normalize='index')

Prediction_KNN,Big-data,Non Big-data
Unsupervised_term_match,,
0,0.321307,0.678693
1,0.956020,0.043980
Total,0.323906,0.676094


In [19]:
# Comparison with SVM
pd.crosstab(df['Unsupervised_term_match'], df['Prediction_SVM'], margins=True, margins_name="Total", normalize='index')

Prediction_SVM,Big-data,Non Big-data
Unsupervised_term_match,,
0,0.016391,0.983609
1,0.665457,0.334543
Total,0.019049,0.980951


In [20]:
# Comparison with Decision Tree
pd.crosstab(df['Unsupervised_term_match'], df['Prediction_DT'], margins=True, margins_name="Total", normalize='index')

Prediction_DT,Big-data,Non Big-data
Unsupervised_term_match,,
0,0.055514,0.944486
1,0.753202,0.246798
Total,0.058371,0.941629


In [21]:
# Comparison with Random Forest
pd.crosstab(df['Unsupervised_term_match'], df['Prediction_RF'], margins=True, margins_name="Total", normalize='index')

Prediction_RF,Big-data,Non Big-data
Unsupervised_term_match,,
0,0.029058,0.970942
1,0.604398,0.395602
Total,0.031414,0.968586


In [22]:
# Comparison with Neural network
pd.crosstab(df['Unsupervised_term_match'], df['Prediction_NN'], margins=True, margins_name="Total", normalize='index')

Prediction_NN,Big-data,Non Big-data
Unsupervised_term_match,,
0,0.075606,0.924394
1,0.722886,0.277114
Total,0.078256,0.921744
